In [6]:
from lib.utils import generate_mean_ensemble_metrics, aggregate_pred_dataframe, generate_mean_ensemble_metrics_auto
import os
import pandas as pd
test_csv_fliles = [file for file in os.listdir('result_cv/esm-33-gearnet-ensemble/fold_0/intermediate') if 'test' in file]
test_csv_fliles.sort()
test_csv_fliles = [f'result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/{file}' for file in test_csv_fliles]

valid_csv_fliles = [file for file in os.listdir('result_cv/esm-33-gearnet-ensemble/fold_0/intermediate') if 'valid' in file]
valid_csv_fliles.sort()
valid_csv_fliles = [f'result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/{file}' for file in valid_csv_fliles]


In [7]:
agg = pd.DataFrame()
for csv_file in test_csv_fliles:
    df = aggregate_pred_dataframe([csv_file])
    me = generate_mean_ensemble_metrics(df)
    print(f'Processing {csv_file}, aggregated metrics: {me["mcc"]})')
    agg = pd.concat([agg, pd.DataFrame([me])], ignore_index=True)

agg.aggregate(['mean', 'std'])


Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_0_test.csv, aggregated metrics: 0.6592)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_1_test.csv, aggregated metrics: 0.6787)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_2_test.csv, aggregated metrics: 0.6558)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_3_test.csv, aggregated metrics: 0.6207)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_4_test.csv, aggregated metrics: 0.6578)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_5_test.csv, aggregated metrics: 0.6491)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_6_test.csv, aggregated metrics: 0.6407)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_7_test.csv, aggregated metrics: 0.6579)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_8_test.csv, aggregated metrics: 0.6457)
Processing

,sensitivity,specificity,accuracy,precision,mcc,micro_auroc
mean,0.602700,0.988620,0.968640,0.747960,0.654480,0.920260
std,0.040115,0.003408,0.001959,0.046113,0.017261,0.013339


In [3]:
# for valid_csv, test_csv in zip(valid_csv_fliles, test_csv_fliles):
#     valid_df = aggregate_pred_dataframe([valid_csv])
#     test_df = aggregate_pred_dataframe([test_csv])
#     me = generate_mean_ensemble_metrics_auto(valid_df, test_df)
#     agg = pd.concat([agg, pd.DataFrame([me])], ignore_index=True)


# agg.aggregate(['mean', 'std'])

In [8]:
import pandas as pd

def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"

# Sample loading of your CSV - this would be replaced with your actual loading
result_df = pd.read_csv('result_cv/result_cv.csv')

metrics = ['mcc', 'sensitivity', 'precision']

grouped = result_df.groupby('model_key').agg({
    k: ['mean', 'std'] for k in metrics
})

record_counts = result_df.groupby('model_key').size()

for metric in metrics:
    grouped[f'{metric}_formatted'] = grouped.apply(lambda row: format_mean_std(row, metric), axis=1)


# Add the record count column to the grouped dataframe
grouped['record_count'] = record_counts.values

formatted_df = grouped[[k + '_formatted' for k in metrics] + ['record_count']]
formatted_df.columns = metrics + ['record_count']
formatted_df = formatted_df.sort_values(metrics[0], ascending=True)

formatted_df = formatted_df.reset_index()
formatted_df

,model_key,mcc,sensitivity,precision,record_count
0,gearnet,0.458 ± 0.024,0.374 ± 0.038,0.619 ± 0.071,5
1,bert,0.487 ± 0.013,0.400 ± 0.038,0.649 ± 0.061,5
2,bert-gearnet,0.521 ± 0.028,0.441 ± 0.095,0.680 ± 0.111,5
3,bert-gearnet-ensemble,0.578 ± 0.018,0.517 ± 0.041,0.691 ± 0.021,5
4,esm-t33,0.649 ± 0.014,0.630 ± 0.039,0.707 ± 0.045,5
5,esm-33-gearnet-ensemble-rus,0.656 ± 0.019,0.747 ± 0.016,0.612 ± 0.044,5
6,esm-33-gearnet,0.657 ± 0.016,0.625 ± 0.013,0.725 ± 0.041,5
7,esm-t33-ensemble,0.681 ± 0.001,0.663 ± 0.010,0.731 ± 0.012,5
8,esm-33-gearnet-ensemble,0.689 ± 0.004,0.659 ± 0.011,0.752 ± 0.018,5
9,esm-33-gearnet-resiboost,0.694 ± 0.009,0.672 ± 0.034,0.748 ± 0.021,5


In [5]:
from tabulate import tabulate
markdown_table = tabulate(formatted_df[['model_key'] + metrics] , headers='keys', tablefmt='pipe', showindex=False)
print(markdown_table)


| model_key                    | mcc           | sensitivity   | precision     |
|:-----------------------------|:--------------|:--------------|:--------------|
| gearnet                      | 0.458 ± 0.024 | 0.374 ± 0.038 | 0.619 ± 0.071 |
| bert                         | 0.487 ± 0.013 | 0.400 ± 0.038 | 0.649 ± 0.061 |
| bert-gearnet                 | 0.521 ± 0.028 | 0.441 ± 0.095 | 0.680 ± 0.111 |
| bert-gearnet-ensemble        | 0.578 ± 0.018 | 0.517 ± 0.041 | 0.691 ± 0.021 |
| esm-t33                      | 0.649 ± 0.014 | 0.630 ± 0.039 | 0.707 ± 0.045 |
| esm-33-gearnet-ensemble-rus  | 0.656 ± 0.019 | 0.747 ± 0.016 | 0.612 ± 0.044 |
| esm-33-gearnet               | 0.657 ± 0.016 | 0.625 ± 0.013 | 0.725 ± 0.041 |
| esm-t33-ensemble             | 0.681 ± 0.001 | 0.663 ± 0.010 | 0.731 ± 0.012 |
| esm-33-gearnet-resiboost-tmp | 0.685 ± 0.004 | 0.659 ± 0.038 | 0.745 ± 0.035 |
| esm-33-gearnet-ensemble      | 0.689 ± 0.004 | 0.659 ± 0.011 | 0.752 ± 0.018 |
| esm-33-gearnet-resiboost  